# 4. Robot sensing

## Introduction

In order to carry out tasks like localization or navigation, a mobile robot has to perceive its workspace. A variety of sensors can be used for that, as well as a number of probabilitic models for managing their behaviour.

## <span style="color:green">OPTIONAL</span>

<span style="color:green">As commented, a number of sensors can be mounted on a mobile robot. In the robotic sensing lecture we discused some of the most popular ones. As an optional exercise, you can look for interesting information about any of them (or any one not listed below) and further describe it here to complete your knowledge.</span>

## 4.1 Sensors for autonomous robots

### 4.1.1 Classification
### 4.1.2 Beacons
#### GPS
### 4.1.3 Range sensors
#### Sonar
#### Infrared
#### Laser scanner
### 4.1.4 Cameras
### 4.1.5 RGB-D cameras

<span style="color:green">***END OF OPTIONAL PART***</span>

## 4.2 Probabilistic sensor models

Typically, the sensors used onboard the robot do not deliver the exact truth of the quantities they are measuring, but a perturbed version. To model this behaviour, sensors' measurements are modelled by the probability distribution $p(z|v)$, where z models the measurmenet and v is the ground truth.

### 4.2.1 Landmark-based models

When the map consist of a collection of landmarks $m={m_i}, i=1,\dots,N$, then the sensors used to provide observations of those landmarks measure:

- **Distance/range** (*e.g.* radio, GPS, etc.): $z_i = d_i = h_i(x,m)+w_i$
- **Bearing** (*e.g.* camera): $z_i = \theta_i = h_i(x,m)+w_i$
- **Distance/range and bearing** (*e.g.* stereo, features in a scan, etc.) $z_i = [d_i,\theta_i]^T = h_i(x,m)+w_i$ *(in this case, $h_i(x,m)$ and $w_i$ are 2D vectors)*

These types of maps and sensor measurements pose a new problem: **data association**, that is, with which landmark $m_i$ correspond the observation $z_i$ to: $h_i(x,m)=h(x,m_i)$. This problem is usually addressed by applying Chi-squared tests, although for the shake of simplicity in this book we will consider it as solved. 

#### Playing with landmarks and robot poses

In the remaining of this section we will familiarise ourselves with the process of observing landmarks from robots located at certain poses, as well as the transformations needed to make use of these observations, that is, to express those observations into the world frame and backwards.

Some relevant concepts:

- **World frame**: $x, y$ coordinates from a selected point of reference $(0, 0)$. We use to keep track of the robots pose, and within the map.
- **Observation**: Information from the real world provided by a sensor, from the point of view (*pov*) of a certain robot.
- **Range-bearing sensor**: Sensor model being used in this lesson. They detect how far is an object $(d)$ and its orientation relative to the robot's $(\theta)$.

The main tools for that are:

- the composition of two poses.
- the composition of a pose and a landmark.
- the propagation of uncertainty through the Jacobians of these compositions.

In [ ]:
#%matplotlib notebook
%matplotlib inline

# IMPORTS

import math
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

from utils.PlotEllipse import PlotEllipse
from utils.DrawRobot import DrawRobot
from utils.tcomp import tcomp
from utils.tinv import tinv, jac_tinv1 as jac_tinv
from utils.Jacobians import J1, J2

We will address several problems of incremental complexity. The following figures will help you to follow the exercise. Concretely:

1. We start with a robot ($R1$) placed at a perflectly known pose $p_1$ in the world frame (no uncertainty at this point), which observes a landmark with a range-bearing (polar) sensor affected by an error characterized with the covarince $W_{1p}$, providing a measurement $z_{1p}$ (In ipolar coordinates). **Our goal is to express the landmark position in the world frame**.
2. In the second point, the robot pose is not perfectly known, but it follows a random variable (RV) so $p_1 \sim N(\overline{p}_1, \Sigma_{p_1}).$. Again, **we have to express the landmark position in the world frame** and analyze what changes.
3. Then, a second robot comes to play, and **we are asked to retrieve the relative pose $p_{12}$ between both robots**.
4. At this point **we predict and obsevation from $R2$ to the landmark**. That is, using the range-bearing observation model and the information of the landmark position and uncertainty as provided by $R1$, we retrieve $z_{2p}$ expressed in polar coordiantes, as well as its associated uncertainty. 
5. Finally, $R2$ takes an observation of the landmark with its own sensor, so now we have two observations of the same landmark, one by $R1$ and another one by $R2$. **How could we combine these pieces of information?**



<center>
<img src="images/fig4-1-1.png" width="800">
</center>

1. **Express the observed landmark in coordinates of the world frame.** Let’s consider a robot R1 at a perfectly known pose $p_1 = [1, 2, 0.5]^T$ which observes
 a landmark $m$ with a range-bearing (polar) sensor affected by a zero-mean
 Gaussian error with covariance $W_{1p} = diag\left([0.25, 0.04]\right)$. The sensor provides the
 measurement $z_{1p} = [4m., 0.7rad.]^T$. 
 
 Compute the Gaussian probability distribution  (mean and covariance) of the landmark observation in the world frame (the same as the robot)  and plot its corresponding ellipse (in magenta, $\sigma=1$). Concretely, you have to complete the ``to_world_frame()`` function, and modify the demo code to show the ellipse representing the uncertainty. 
 
 Consider the following:
 
 - You can express a sensor measurement in polar coordinates ($z_p=[r,\alpha]^T$) as cartesian coordinates ($z_c=[z_x,z_y]^T)$ by:
 
 $$
     z_c = \begin{bmatrix} z_x \\ z_y \end{bmatrix} 
         = \begin{bmatrix} r \ cos\alpha \\ r \ sin\alpha \end{bmatrix} 
         = f(r,\alpha)
 $$
 
 - While computing the covariance of the landmark observation, you have to start by computing the covariance of the observation in the Cartesian robot $R1$ frame. That is:
 
 $$  
 W_{c} = \frac{\partial f(z_x,z_y)}{\partial \{r,\alpha\}} \ W_{p} \ \frac{\partial f(z_x,z_y)}{\partial \{r,\alpha\}}^T
 $$
   
   Then you can get the convariance in the world frame as:
   
   $$ W_{z\_w} = \frac{\partial f(p,z_c)}{\partial p} \ Q_{p1\_w} \ \left( \frac{\partial f(p,z_c)}{\partial p} \right)^T +
        \frac{\partial f(p,z_c)}{\partial z_c} \ W_{c} \ \left( \frac{\partial f(p,z_c)}{\partial z_c} \right)^T
   $$
   
   where $f(p,z_c) = p \oplus z_c$, that is, the composition of the pose and the landmark.
   
   - Note that $\frac{\partial f(p,z_c)}{\partial p}$ and $\frac{\partial f(p,z_c)}{\partial z_c}$ are the same Jacobians as previously used to compose two poses in *robot motion*, but with a reduced size since **while working with landmarks the orientation is meaningless, only the position matters**. The functions ``J1()`` and ``J2()`` implement these jacobians for you.
  

In [ ]:
def to_world_frame(p1_w, Qp1_w, z1_p_r, W1):
    """ Covert the observation z1_p_r to the world frame
    
        Args:
            p1_w: Pose of the robot(in world frame)
            Qp1_w: Covariance of the robot
            z1_p_r: Observation to a landmark (polar coordinates) from robots perspective
            W1: Covariance of the sensor in polar coordinates
    
        Returns:
            z1_w: Pose of landmark in the world frame
            Wz1: Covariance associated to z1_w
    """
    
    # Definition of useful variables
    r, a = z1_p_r[0,0], z1_p_r[1,0]
    s, c = np.sin(a), np.cos(a)

    # Jacobian to convert the measurement uncertainty from polar to cartesian coordinates
    Jac_pol_car = np.array([
        [None, None],
        [None, None]
    ])

    # Built a tuple with:
    # z1_car_rel[0]: coordinates of the sensor measurement in cartesian coordinates relative to robot position
    # z1_car_rel[1]: its associated uncertainty expressed in cartesian coordinates
    z1_car_rel = (
            np.vstack([None,None]), # position
            None@W1@None # uncertainty
            )
    
    z1_ext = np.vstack([z1_car_rel[0], 0]) # Extends z1 for its usage in the Jacobian functions J1 and J2

    # Build the jacobians 
    Jac_ap = J1(p1_w ,z1_ext)[0:2,:] # Jacobian for expressing the uncertainty in the robot pose in a global frame
    Jac_aa = J2(p1_w ,z1_ext)[0:2,0:2] # This one expresses the uncertainty in the measurment in a global frame
    
    z1_w = tcomp(None ,None)[0:2,[0]] # Compute coordinates of the landmark in the world
    Wz1 = (None @ Qp1_w @ None
          + None @ z1_car_rel[1] @ None) # Finally, propagate the covariance!
    
    return z1_w, Wz1

In [ ]:
# Robot
p1_w = np.vstack([1, 2, 0.5]) # Robot R1 pose
Qp1_w = np.zeros((3, 3)) # Robot pose convariance matrix (uncertainty)

# Landmark observation
z1_p_r = np.vstack([4., .7]) # Measurement/Observation
W1 = np.diag([0.25, 0.04]) # Sensor noise covariance

# Express the landmark observation in the world frame (mean and covariance)
z1_w, Wz1 = to_world_frame(p1_w, Qp1_w, z1_p_r, W1)

# Visualize the results
fig, ax = plt.subplots()
plt.xlim([-.5, 10])
plt.ylim([-.5, 10])
plt.grid()
plt.tight_layout()

DrawRobot(fig, ax, p1_w, label='R1', color='blue')
    
ax.plot(z1_w[0, 0], z1_w[1, 0], 'o', label='Z1', color='green')
PlotEllipse(fig, ax, None, None, color='green')

plt.legend()
print('----\tExercise 4.1.1\t----\n'+
      'z1_w = {}\'\n'.format(z1_w.flatten())
      + 'Wz1_w = \n{}\n'.format(Wz1))

<span style="color:blue">Expected results for demo:</span>


```
---- Exercise 4.1.1 ----
z1_w = [2.44943102 5.72815634]'
Wz1_w = 
[[ 0.58879177 -0.13171532]
 [-0.13171532  0.30120823]]
```

2. **Adding uncertainty to the robot posiion.** Now, let’s assume that the robot pose is not known, but it is a RV that follows a
Gaussian probability distribution: $p_1 \sim N([1, 2, 0.5]^T, \Sigma_1)$ with $\Sigma_1 = diag([0.08,0.6,
0.02 ])$.

    1. Compute the covariance matrix $\Sigma_{m1}$ of the landmark in the world frame and
     plot it as an ellipse centered at the mean $m_1$ (in blue, $sigma= 1$). Plot also
     the covariance of the robot pose (in blue, $sigma= 1$).
     
    2. Compare the covariance with that obtained in the previous case. Is it
     bigger? Is it bigger than that of the robot? Why?  **<span style="color:orangered">Comment this in the Student discussion</span>**
     
     


In [ ]:
# Robot
p1_w = np.vstack([1, 2, 0.5]) # Robot R1 pose
Qp1_w = np.diag([0.08,0.6,0.02])  # Robot pose convariance matrix (uncertainty)

# Landmark observation
z1_p_r = np.vstack([4., .7]) # Measurement/Observation
W1 = np.diag([0.25, 0.04]) # Sensor noise covariance

# Express the landmark observation in the world frame (mean and covariance)
z1_w, Wz1 = to_world_frame(p1_w, Qp1_w, z1_p_r, W1)

# MATPLOTLIB
fig, ax = plt.subplots()
plt.xlim([-.5, 10])
plt.ylim([-.5, 10])
plt.grid()
plt.tight_layout()

fig.canvas.draw()

DrawRobot(fig, ax, p1_w, label='R1', color='red')  
PlotEllipse(fig, ax, None, None, color='blue')

ax.plot(z1_w[0, 0], z1_w[1, 0], 'o', label='Z1', color='green')
PlotEllipse(fig, ax, None, None, color='green')

plt.legend()
print('---- Exercise 4.1.2 ----\n'+
      'Wz1_w = \n{}\n'.format(Wz1))


<span style="color:blue">Expected results for demo:</span>

```
---- Exercise 4.1.2 ----
Wz1_w = 
[[ 0.94677477 -0.23978943]
 [-0.23978943  0.94322523]]
```

3. **Getting the relative pose between two robots**. Another robot `R2` is at pose $p_2 \sim ([6m., 4m., 2.1rad.]^T, \Sigma_2)$ with $\Sigma_2 = diag([0.20,0.09,
 0.03])$. Plot `p2` and its ellipse (covariance) in green ($sigma=1$). Compute the relative
 pose `p12` between `R1` and `R2`. 
 
 This relative pose can be obtained in two different ways:
 - Through the composition of poses, but using $\ominus p1$ instead of $p1$. Implement it in ``inverse_composition1()``.
 
 Mean: 
 
 $$
 p12 = \ominus p1 \oplus p2 = f(\ominus p1, p2) = 
 \begin{bmatrix} 
  x_{\ominus p1} + x_{p2} cos \theta_{\ominus p1} - y_{p2} sin \theta_{\ominus p1} \\
  y_{\ominus p1} + x_{p2} sin \theta_{\ominus p1} + y_{p2} cos \theta_{\ominus p1} \\
  \theta_{\ominus p1} + \theta_{p2}
 \end{bmatrix}
 $$
 
 Covariance:
 
 $$
 \Sigma_{p12} = \frac{\partial p12}{\partial \ominus p1} \frac{\ominus p1}{\partial p1} \Sigma_{p1} \frac{\ominus p1}{\partial p1}^T \frac{\partial p12}{\partial \ominus p1}^T 
 +
 \frac{\partial p12}{\partial p2} \Sigma_{p2}  \frac{\partial p12}{\partial p2}^T
 \\
 \text{Applying the Chain rule} \rightarrow \Sigma_{p12} = \frac{\partial p12}{\partial \ominus p1} \Sigma_{\ominus p1} \frac{\partial p12}{\partial \ominus p1}^T
 +
 \frac{\partial p12}{\partial p2} \Sigma_{p2}  \frac{\partial p12}{\partial p2}^T
 $$
 
 Being:
 
 $$
 \frac{\partial p12}{\partial \ominus p1} = 
 \begin{bmatrix}
 1 & 0 & -x_{p2} sin \theta_{\ominus p1} - y_{p2} cos \theta_{\ominus p1} \\
 0 & 1 & x_{p2} cos \theta_{\ominus p1} - y_{p2} sin \theta_{\ominus p1} \\
 0 & 0 & 1
 \end{bmatrix}  
 \; \; \; \; \; \frac{\partial p12}{\partial p2} = 
 \begin{bmatrix}
cos \theta_{\ominus p1} & -sin \theta_{\ominus p1} & 0\\
sin \theta_{\ominus p1} & cos \theta_{\ominus p1} & 0\\
 0 & 0 & 1
 \end{bmatrix}
 \\
 \frac{\partial \ominus p1}{\partial p1} = 
 \begin{bmatrix}
 -cos \theta_{p1} & -sin \theta_{p1} & x_{p1} sin \theta_{p1} - y_{p1} cos \theta_{p1} \\
 sin \theta_{p1} & -cos \theta_{p1} & x_{p1} cos \theta_{p1} + y_{p1} sin \theta_{p1}\\
 0 & 0 & -1 \\
 \end{bmatrix}
 \; \; \; \; \; \Sigma_{\ominus p1} = \frac{\partial \ominus p1}{\partial p1} \Sigma_{p1} \frac{\partial \ominus p1}{\partial p1}^T
 $$ 
 
 - Using the inverse composition of poses. This one is given for you in ``inverse_composition2()``.

In [ ]:
def inverse_composition1(p1_w, Qp1_w, p2_w, Qp2_w):
    jac_inv_p = jac_tinv(p1_w)

    inv_r1 = (
        tinv(p1_w),
        jac_inv_p @ Qp1_w @ jac_inv_p.T
    )

    jac_p12_inv = J1(inv_r1[0], p2_w)
    jac_p12_p2 = J2(inv_r1[0], p2_w)

    p12_w = tcomp(None, None)
        
    Qp12_w = (
            None@inv_r1[1]@None 
            + None@Qp2_w@None
        )
    
    return p12_w, Qp12_w

In [ ]:
def inverse_composition2(p1_w, Qp1_w, p2_w, Qp2_w):
    dx, dy = p2_w[0, 0]-p1_w[0, 0], p2_w[1, 0]-p1_w[1, 0]
    a = p2_w[2, 0] - p1_w[2, 0]
    c, s = np.cos(p1_w[2, 0]), np.sin(p1_w[2, 0])

    p12_w = np.array([
        [dx*c + dy*s],
        [-dx*s + dy*c],
        [a]])
    
    jac_p12_r1 = np.array([
        [-c, -s, -dx*s + dy*c],
        [s, -c, -dx*c - dy*s],
        [0, 0, -1]
    ])

    jac_p12_r2 = np.array([
        [c, s, 0],
        [-s, c, 0],
        [0, 0, -1]
    ])

    #jac_p1_pinv = np.linalg.inv(jac_tinv(r1[0]))

    Qp12_w = jac_p12_r1@Qp1_w@jac_p12_r1.T + jac_p12_r2@Qp2_w@jac_p12_r2.T

    return p12_w, Qp12_w

In [ ]:
# Robot R1
p1_w = np.vstack([1., 2., 0.5])
Qp1_w = np.diag([0.08, 0.6, 0.02])

# Robot R2
p2_w = np.vstack([6., 4., 2.1])
Qp2_w = np.diag([0.20, 0.09, 0.03])

# Obtain the relative pose p12 between both robots through the composition of poses
p12_w, Qp12_w = inverse_composition1(p1_w, Qp1_w, p2_w, Qp2_w)
print( '----\tExercise 4.1.3 with method 1\t----\n'+
        'p12_w = {}\'\n'.format(p12_w.flatten())+
        'Qp12_w = \n{}\n'.format(Qp12_w))

# Obtain the relative pose p12 between both robots through the inverse composition of poses
p12_w, Qp12_w = inverse_composition2(p1_w, Qp1_w, p2_w, Qp2_w)
print( '----\tExercise 4.1.3 with method 2\t----\n'+
        'p12_w = {}\'\n'.format(p12_w.flatten())+
        'Qp12_w = \n{}\n'.format(Qp12_w))

<span style="color:blue">Expected results:</span>
 ```
 p12_w = [ 5.34676389 -0.64196257  1.6       ]'
 
 Qp12_w = 
 [[0.38248035 0.24115    0.01283925]
 [0.24115    1.16751965 0.10693528]
 [0.01283925 0.10693528 0.05      ]]
 ```

4. **Predicting an observation from the second robot.** According to the information, provided by R1, that we have about the position of the landmark m in
 the world coordinates (its location $z_{1\_w}$ and its associated uncertainty $W_{z_1\_w}$), compute the *predicted observation*
 distribution of $z_{2p} =[r, \alpha] \sim N(z_{2p}, W_{2p})$ as taken by a range-bearing sensor mounted on `R2`.
 Consider the following:
 
 - The range-bearing model for taking measurements is *(Notes: use ``np.arctan2()`` for computing the angle. At this point, ignore the noise $w_i$)*:
 
 $$
 z_i = \begin{bmatrix} r_i \\ \alpha_i \end{bmatrix} = h(x,m_i) + w_i = 
 \begin{bmatrix} \sqrt((x_i-x)^2+(y_i-y)^2) \\ atan(\frac{y_i-y}{x_i-x}) - \theta \end{bmatrix} 
 + w_i
 $$
 
 - We need to compute the covariance of the predicted observation in Polar  coordinates $(W_{2p})$. For that, use the following Jacobian: 
 $$
     \frac{\partial{p}}{\partial{c}} = 
     \begin{bmatrix}
         \cos{(\alpha+\theta)}  & \sin{(\alpha+\theta)} \\
         -\sin{(\alpha+\theta)} / r  & \cos{(\alpha+\theta)} / r
     \end{bmatrix}
 $$

In [ ]:
def predicted_obs_from_pov(p1_w, z1_w, Wz1_w):
    """ Method to translate a pose+covariance in the world frame to an observation.
    
        This method only translated the landmark to the pov of the robot.
        It does not simulate a new observation.
        
        Args:
            p1_w: Pose of the robot which acts as pov
            z1_w: Landmark observed in cartesian coordinates(world frame)
            Wz1_w: Covariance associated to the landmark.
        Returns:
            z2_pr: Expected observation of z1 from pov of p1_w
            W2_p: Covariance associated to z2_pr
    """

    # Take a measurement using the range-bearing model
    z2_pr = np.vstack([
            None, # distance
            None # angle
        ])
      
    # Jacobian from cartesian to polar at z2p_r when the covariance is in
    # global coordianes
    theta = z2_pr[1, 0] + p1_w[2, 0] 
    s, c = np.sin(theta), np.cos(theta)
    r = z2_pr[0, 0]

    Jac_car_pol = np.array([
        [None, None],
        [None, None]
    ])

    # Finally, propagate the uncertainty to polar coordinates in the
    # robot frame
    W2_p = None@None@None
    
    return z2_pr, W2_p

In [ ]:
p2_w = np.vstack([6., 4., 2.1])

z2_pr, W2_p = predicted_obs_from_pov(p2_w, z1_w, Wz1)
print( '---- Exercise 4.1.4 ----\n'+
    'z2p_r = {}\'\n'.format(z2_pr.flatten())+
    'W2_p = \n{}\n'.format(W2_p)    
)

<span style="color:blue">Expected output:</span>
```
---- Exercise 4.1.4 ----
z2p_r = [3.94880545 0.58862004]'
W2_p = 
[[ 1.13481128 -0.03710981]
 [-0.03710981  0.04843106]]
```

5. **Combining observations of the same landmark.** Assume now that a measurement $z_2 = [4 m., 0.3 rad.]^T$ of the landmark is taken
 from R2 with a sensor having the same precision as that of R1 ($W_{2p}= W_{1p}$)
 
 1. What is the pdf of the observed landmark according to this observation? Plot the corresponding ellipse (in green, sigma=1).
 2. Two different pdf’s are now associated to the same landmark.
    1. Is that a contradiction?
    2. Can you work out a solution that combines these two “pieces of
     information”? Plot it in red. **<span style="color:orangered">Comment this in the Student discussion</span>**
     
    
<figure style="text-align:center">
  <img src="images/fig4-1-2.png" width="400" alt="">
  <figcaption>Fig. 2: Results from the last exercise </figcaption>
</figure>    

In [ ]:
def combine_pdfs(z1_w, Wz1_w, z2_w, Wz2_w):
    """ Method to combine the pdfs associated with two observations of the same landmark.  
        
        Args:            
            z1_w: Landmark observed in cartesian coordinates(world frame) from Robot 1
            Wz1_w: Covariance associated to the landmark.
            z1_w: Landmark observed in cartesian coordinates(world frame) from Robot 2
            Wz2_w: Covariance associated to the landmark.
        Returns:
            z: Combined observation
            W_z: Uncertainty associated to z
    """
    None # Implement the needed code here

    W_z = None
    z = None

    return z, W_z
    

In [ ]:
z2_p_r = np.vstack([4., .3])
Wz2_p_r = np.diag([0.25, 0.04])

z1_w, Qz1 = to_world_frame(p1_w, Qp1_w, z1_p_r, W1)
z2_w, Qz2 = to_world_frame(p2_w, Qp2_w, z2_p_r, W1)

# Show results
fig, ax = plt.subplots()
plt.xlim([-.5, 10])
plt.ylim([-.5, 10])
plt.grid()
plt.tight_layout()

fig.canvas.draw()

DrawRobot(fig, ax, p1_w, label='R1', color='blue')
PlotEllipse(fig, ax, p1_w, Qp1_w, color='blue')

DrawRobot(fig, ax, p2_w, label='R2', color='green')
PlotEllipse(fig, ax, p2_w, Qp2_w, color='green')
   
ax.plot(z1_w[0, 0], z1_w[1, 0], 'o', label='Z1', color='blue')
PlotEllipse(fig, ax, z1_w, Qz1, color='blue')
          
ax.plot(z2_w[0, 0], z2_w[1, 0], 'o', label='Z2', color='green')
PlotEllipse(fig, ax, z2_w, Qz2, color='green')

z_w, Wz_w = combine_pdfs(z1_w, Qz1, z2_w, Qz2)
ax.plot(z_w[0, 0], z_w[1, 0], 'o', label='Z3', color='red')
PlotEllipse(fig, ax, z_w, Wz_w, color='red')
          
plt.legend()

# Print results
print( '----\tExercise 4.1.5\t----\n'+
    'z2_w = {}\'\n'.format(z2_w.flatten())+
    'Qz2 = \n{}\n'.format(Qz2)
    )

# Print results
print( '----\tExercise 4.1.5 parte 2\t----\n'+
    'z_w = {}\'\n'.format(z_w.flatten())+
    'Wz_w = \n{}\n'.format(Wz_w)
    )

<span style="color:blue">Expected ouputs:</span>

### Sensor measurement from R2

```
z2_w = [3.05042514 6.70185272]'
Qz2 = 
[[0.84693794 0.4333316 ]
 [0.4333316  0.81306206]]
```

### Combined information
```
---- Exercise 4.1.5 parte 2 ----
z_w = [2.58757252 6.15534036]'
Wz_w = 
[[0.37966125 0.07773125]
 [0.07773125 0.36999739]]
```

## <span style="color:green">OPTIONAL</span>

<span style="color:green">An alternative to *landmark observation models* are *scan observation* ones, which work with scan-based sensors. Below, the three most popular ones are listed. Surf the internet for some code illustrating any of them, and include it in the notebook with a brief description of how it works and its purpose.</span>

### 4.2.2 Scan observation models

Scan observation models are used when the sensor mounted on the robot provides a scan measuring distance and angle to obstacles in the workspace, *e.g.* a laser range finder. In this case, each element in the map is a cell described by its position (and probably a color representing if its free of obstacles or occupied), and data association is not explicitly addressed. 

#### Beam model
#### Likelihood field
#### Scan matching

<span style="color:green">***END OF OPTIONAL PART***</span>

## Student discussion
In the cell below, discuss what has been done in the notebook, what you have found interesting, or any other relevant thought.

<span style="color:orangered">***Write your answer here***</span>